Customer churn prediction is to measure why customers are leaving a business. In this tutorial we will be looking at customer churn in Bank business. We will build a deep learning model to predict the churn and use precision, recall, f1-score to measure performance of our model


In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

LOAD THE DATA

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


DROP UNNECCESSARY COLUMNS

In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


FEATURE ENGINEERING

In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


ONE HOT ENCODING CATEGORICAL VALUES

In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


SCALING THE DATASET

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

SPLITTING THE DATASET INTO TRAINING AND TEST SET

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [26]:
X_train.shape

(8000, 13)

In [27]:
X_test.shapeb

(2000, 13)

IMPORTING TENSORFLOW LIBRARIES

In [18]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

BUILD THE MODEL(ANN)

In [33]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [34]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 2s 854us/step - loss: 0.5041 - accuracy: 0.7944
Epoch 2/100
250/250 [==============================] - 0s 718us/step - loss: 0.4742 - accuracy: 0.7964
Epoch 3/100
250/250 [==============================] - 0s 677us/step - loss: 0.4673 - accuracy: 0.7972
Epoch 4/100
250/250 [==============================] - 0s 730us/step - loss: 0.4588 - accuracy: 0.7981
Epoch 5/100
250/250 [==============================] - 0s 851us/step - loss: 0.4508 - accuracy: 0.8046
Epoch 6/100
250/250 [==============================] - 0s 687us/step - loss: 0.4417 - accuracy: 0.8111
Epoch 7/100
250/250 [==============================] - 0s 671us/step - loss: 0.4337 - accuracy: 0.8155
Epoch 8/100
250/250 [==============================] - 0s 727us/step - loss: 0.4280 - accuracy: 0.8186
Epoch 9/100
250/250 [==============================] - 0s 667us/step - loss: 0.4237 - accuracy: 0.8158
Epoch 10/100
250/250 [==============================] - 0s 679us/step - l

As We See, the precision, recall and f1 score of Class 1 is very low due to imbalanced dataset

Mitigating Skewdness of Data

Method 1: Undersampling

reference: https: // www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets


In [36]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [40]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [41]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [42]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [44]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6774 - accuracy: 0.5802
Epoch 2/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6464 - accuracy: 0.6349
Epoch 3/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6310 - accuracy: 0.6542
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6239 - accuracy: 0.6646
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6175 - accuracy: 0.6720
Epoch 6/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6120 - accuracy: 0.6744
Epoch 7/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6056 - accuracy: 0.6793
Epoch 8/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5993 - accuracy: 0.6870
Epoch 9/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5936 - accuracy: 0.6910
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5881 - accura

As we see, there is considerable Improvement in the f1, recall and precision scores of Class 1 Value.
The f1 score has improved from 0.58 to 0.77.

Method2: Oversampling


In [45]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exitedb.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [46]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [47]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 919us/step - loss: 0.6620 - accuracy: 0.6154
Epoch 2/100
399/399 [==============================] - 0s 920us/step - loss: 0.6135 - accuracy: 0.6735
Epoch 3/100
399/399 [==============================] - 0s 900us/step - loss: 0.5829 - accuracy: 0.7017
Epoch 4/100
399/399 [==============================] - 0s 857us/step - loss: 0.5554 - accuracy: 0.7208
Epoch 5/100
399/399 [==============================] - 0s 799us/step - loss: 0.5319 - accuracy: 0.7394
Epoch 6/100
399/399 [==============================] - 0s 747us/step - loss: 0.5101 - accuracy: 0.7530
Epoch 7/100
399/399 [==============================] - 0s 829us/step - loss: 0.4970 - accuracy: 0.7612
Epoch 8/100
399/399 [==============================] - 0s 862us/step - loss: 0.4893 - accuracy: 0.7596
Epoch 9/100
399/399 [==============================] - 0s 777us/step - loss: 0.4855 - accuracy: 0.7601
Epoch 10/100
399/399 [==============================] - 0s 792us/step - l

f1-score for minority class 1 improved from 0.58 to 0.79.


Method3: SMOTE

To install imbalanced-learn library use pip install imbalanced-learn command

In [50]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [56]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

0    7963
1    7963
Name: Exited, dtype: int64

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [58]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 611us/step - loss: 0.6671 - accuracy: 0.5758
Epoch 2/100
399/399 [==============================] - 0s 621us/step - loss: 0.6068 - accuracy: 0.6684
Epoch 3/100
399/399 [==============================] - 0s 586us/step - loss: 0.5796 - accuracy: 0.6971
Epoch 4/100
399/399 [==============================] - 0s 596us/step - loss: 0.5640 - accuracy: 0.7059
Epoch 5/100
399/399 [==============================] - 0s 596us/step - loss: 0.5538 - accuracy: 0.7171
Epoch 6/100
399/399 [==============================] - 0s 679us/step - loss: 0.5424 - accuracy: 0.7272
Epoch 7/100
399/399 [==============================] - 0s 641us/step - loss: 0.5301 - accuracy: 0.7365
Epoch 8/100
399/399 [==============================] - 0s 597us/step - loss: 0.5195 - accuracy: 0.7414
Epoch 9/100
399/399 [==============================] - 0s 570us/step - loss: 0.5094 - accuracy: 0.7480
Epoch 10/100
399/399 [==============================] - 0s 638us/step - l

SMOT Oversampling increases f1 score of minority class 1 from 0.58 to 0.81.


Method4: Use of Ensemble with undersampling

In [62]:
df1.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [65]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [67]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [68]:
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,Exited
5710,0.856,0.216216,0.5,0.554265,0.333333,0,0,0.339721,1,0,0,0,1,0
3745,0.852,0.256757,0.1,0.371163,0.333333,1,1,0.980432,0,1,0,1,0,0
5429,0.664,0.405405,0.7,0.000000,0.333333,1,0,0.325318,1,0,0,1,0,0
551,0.648,0.391892,0.6,0.426077,0.000000,1,1,0.010339,0,1,0,0,1,1
8967,0.970,0.094595,0.7,0.000000,0.333333,1,1,0.417230,1,0,0,0,1,0


In [70]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [71]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [72]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 866us/step - loss: 0.6631 - accuracy: 0.6243
Epoch 2/100
98/98 [==============================] - 0s 907us/step - loss: 0.6369 - accuracy: 0.6493
Epoch 3/100
98/98 [==============================] - 0s 804us/step - loss: 0.6259 - accuracy: 0.6518
Epoch 4/100
98/98 [==============================] - 0s 866us/step - loss: 0.6186 - accuracy: 0.6605
Epoch 5/100
98/98 [==============================] - 0s 773us/step - loss: 0.6107 - accuracy: 0.6650
Epoch 6/100
98/98 [==============================] - 0s 815us/step - loss: 0.6043 - accuracy: 0.6765
Epoch 7/100
98/98 [==============================] - 0s 804us/step - loss: 0.5949 - accuracy: 0.6854
Epoch 8/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5863 - accuracy: 0.6906
Epoch 9/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5769 - accuracy: 0.7050
Epoch 10/100
98/98 [==============================] - 0s 835us/step - loss: 0.5658 - accuracy: 

In [73]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
98/98 [==============================] - 1s 1ms/step - loss: 0.6884 - accuracy: 0.5488
Epoch 2/100
98/98 [==============================] - 0s 938us/step - loss: 0.6573 - accuracy: 0.6077
Epoch 3/100
98/98 [==============================] - 0s 980us/step - loss: 0.6390 - accuracy: 0.6320
Epoch 4/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6256 - accuracy: 0.6406
Epoch 5/100
98/98 [==============================] - 0s 1ms/step - loss: 0.6156 - accuracy: 0.6630
Epoch 6/100
98/98 [==============================] - 0s 918us/step - loss: 0.6066 - accuracy: 0.6749
Epoch 7/100
98/98 [==============================] - 0s 866us/step - loss: 0.5996 - accuracy: 0.6835
Epoch 8/100
98/98 [==============================] - 0s 876us/step - loss: 0.5937 - accuracy: 0.6906
Epoch 9/100
98/98 [==============================] - 0s 1ms/step - loss: 0.5910 - accuracy: 0.6893
Epoch 10/100
98/98 [==============================] - 0s 990us/step - loss: 0.5856 - accuracy: 0.69

In [74]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
87/87 [==============================] - 1s 861us/step - loss: 0.6886 - accuracy: 0.5347
Epoch 2/100
87/87 [==============================] - 0s 839us/step - loss: 0.6398 - accuracy: 0.6484
Epoch 3/100
87/87 [==============================] - 0s 1ms/step - loss: 0.6135 - accuracy: 0.6704
Epoch 4/100
87/87 [==============================] - 0s 885us/step - loss: 0.5996 - accuracy: 0.6827
Epoch 5/100
87/87 [==============================] - 0s 861us/step - loss: 0.5903 - accuracy: 0.6870
Epoch 6/100
87/87 [==============================] - 0s 849us/step - loss: 0.5821 - accuracy: 0.6971
Epoch 7/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5762 - accuracy: 0.6982
Epoch 8/100
87/87 [==============================] - 0s 2ms/step - loss: 0.5701 - accuracy: 0.7079
Epoch 9/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5646 - accuracy: 0.7094
Epoch 10/100
87/87 [==============================] - 0s 1ms/step - loss: 0.5586 - accuracy: 0.7130

In [75]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [76]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.94      0.73      0.82      1593
           1       0.44      0.81      0.57       407

    accuracy                           0.75      2000
   macro avg       0.69      0.77      0.69      2000
weighted avg       0.83      0.75      0.77      2000



f1-score for minority class 1 is 0.57